### Data Cleaning

In [1]:
from research.sandbox.url_deduplication import remove_similar_rows as rsr
import pandas as pd
import os
import re

# Folder path
folder = 'raw_data'

# List all files in the folder and filter the .tsv.xz files
file = [f for f in os.listdir(folder) if f.endswith('.tsv.xz')]

# Assuming there's only one file, get the full file path
file_path = os.path.join(folder, file[0])

# Read the CSV file
df = pd.read_csv(file_path, sep='\t', compression='xz')

# Clean column names (remove extra spaces)
df.columns = df.columns.str.strip()

# Function to clean text by removing HTML tags, URLs, and extra spaces
def clean_text(text):
    if pd.isna(text):  # Return empty string if value is NaN
        return ''
    text = re.sub(r'<[^>]+>', ' ', text)  # Remove HTML tags
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)  # Remove URLs
    text = re.sub(r'&[a-zA-Z0-9#]+;', ' ', text)  # Remove HTML entities
    text = re.sub(r'\s+', ' ', text).strip()  # Reduce multiple spaces
    return text

df["content"] = df["content"].apply(clean_text)

df


,id,pubtime,medium_code,medium_name,char_count,head,article_link,content_id,content
0,56544211,2025-04-03 06:00:00+02,SWII,swissinfo.ch,8674,Dieser Schweizer managt grosse Hotels in China,https://www.swissinfo.ch/ger/swiss-abroad/dies...,0015b706-80dc-d59f-623b-15b565b47373,Elisa Tirabassi Carlo Schmed (rechts) in seine...
1,56524029,2025-04-01 12:03:00+02,SRFA,srf Audio,6029,Mehr Patroullien unterwegs: neue Struktur für ...,https://www.srf.ch/play/embed?urn=urn:srf:audi...,00349580-ebd5-2e83-942b-6dfb58132312,"Die Basler Polizei hat sich neu organisiert, m..."
2,56526108,2025-04-01 20:11:45+02,ZSZO,zsz.ch,5018,Pro und Kontra: Sollen AKW unter Denkmalschutz...,https://www.zsz.ch/pro-contra-340993213033,0036302d-cce5-36fb-1280-0ef8c2d3c33e,Das Zeitalter der Atomkraft neigt sich dem End...
3,56517906,2025-04-01 00:00:00+02,BT,Badener Tagblatt,7744,Sechs Kirchgemeinden wollen fusionieren,NaN,003ba585-9050-2084-991e-8aedb3a14ef5,Das Kirchspiel und das Untere Aaretal entschei...
4,56537321,2025-04-03 00:00:00+02,WOZ,Die Wochenzeitung,15788,Ehe für die ethnische Einheit,NaN,00469c2f-9da6-12e0-7049-6e9c1a2adc05,Von Enrico Kampmann (Text) und Ursula Häne (Fo...
...,...,...,...,...,...,...,...,...,...
13262,56551993,2025-04-04 00:00:00+02,TZ,Thurgauer Zeitung,1656,Mut machen und Positives betonen,NaN,ffde67d1-9d0e-227a-7e49-18597eb574c9,"«Salzkorn», Ausgabe vom 26. März Lieber pex so..."
13263,56519378,2025-04-01 00:00:00+02,BU,Der Bund,1861,Marine Le Pen hat sich selbst disqualifiziert,NaN,fff0ca75-fb9c-32ff-84ef-d0340390593d,Oliver Meiler Frankreich ist die Wiege der Gew...
13264,56519753,2025-04-01 00:00:00+02,ZWA,20 minuten,628,Skitourengänger stirbt nach Abfahrt,NaN,fff4175a-15ae-fa15-6429-f313c05bbf08,ZERMATT Ein Skitourengänger stürzte in der Reg...
13265,54514894,2025-04-01 08:05:00+02,BLIO,blick.ch,7696,Raus aus den Schulden – Experte ordnet Tipps e...,https://www.blick.ch/wirtschaft/sparexperte-or...,fff6f293-39f3-e3ef-1e73-12f5586358b9,Sparen will gelernt sein: Blick stellt dir ach...


In [2]:

# Drop same or nearly same articles
df = rsr(df, 0.98)

# make the pubtime a df datetime format
df['pubtime'] = pd.to_datetime(df['pubtime'])

# Create 'cleaned_data' folder if it doesn't exist
output_folder = 'cleaned_data'
os.makedirs(output_folder, exist_ok=True)

# TODO aktuell zu langsam allfällige unnötige Spalten entfernen
# Save as Parquet-File in the cleaned_data folder
output_file = os.path.join(output_folder, "cleaned_data.parquet")
df.to_parquet(output_file, engine="pyarrow", index=False)

In [3]:
df

,id,pubtime,medium_code,medium_name,char_count,head,article_link,content_id,content,url,base_link
0,56554084,2025-04-04 06:47:53+02:00,NNTA,tagesanzeiger.ch,13136,Rundgang in 9 Stationen: «Über diesen Raum kur...,https://www.tagesanzeiger.ch/50-jahre-microsof...,0ffe955b-9fe4-990c-ff02-45e126e307a4,Ein Mann in einem blauen Pullover erklärt etwa...,https://www.tagesanzeiger.ch/50-jahre-microsof...,NaN
1,56532802,2025-04-02 09:43:27+02:00,BEOLO,berneroberlaender.ch,1451,88. Schweizer Landesrundfahrt: Eine Tour de Su...,https://www.berneroberlaender.ch/88-tour-de-su...,d38648c0-2fa3-bb38-b833-ad7e95446a9b,Das Radspektakel im Sommer bietet eine anspruc...,https://www.berneroberlaender.ch/88-tour-de-su...,NaN
2,56520060,2025-04-01 06:00:00+02:00,BEOLO,berneroberlaender.ch,8367,Gebühren auf der Aare: Inspiriert von Iseltwal...,https://www.berneroberlaender.ch/aareboeoetle-...,391e65a4-9b6b-9e94-6bb5-65b7db020ca8,Zwei Personen in einem Schlauchboot starten ei...,https://www.berneroberlaender.ch/aareboeoetle-...,NaN
3,56525628,2025-04-01 18:01:34+02:00,NNBU,derbund.ch,5671,Cyberangriff auf Autozulieferer: Wie Hacker ei...,https://www.derbund.ch/adval-tech-wie-hacker-d...,454158d1-3aeb-171c-8838-4ea671f971a2,Adval Tech kämpfte bereits mit der Krise europ...,https://www.derbund.ch/adval-tech-wie-hacker-d...,NaN
4,56536671,2025-04-02 19:53:10+02:00,NNTA,tagesanzeiger.ch,5962,Ausbau der AHV: Nach der 13. AHV-Rente der näc...,https://www.tagesanzeiger.ch/ahv-rente-pension...,07be26ae-05ed-a76d-57c1-bc755d26e3ab,Eine Mitte-links-Allianz will die AHV weiter a...,https://www.tagesanzeiger.ch/ahv-rente-pension...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9140,56538595,2025-04-03 00:00:00+02:00,TVS,TV Star,4232,“ER SOLL SEINE TATEN BÜSSEN”,NaN,07f7c51f-b8a7-2ca9-df2f-9baed220e69b,«HOW TO SELL DRUGS ONLINE (FAST)» ENDET FURIOS...,“ER SOLL SEINE TATEN BÜSSEN”,no_link
9141,56538584,2025-04-03 00:00:00+02:00,TVS,TV Star,14341,"“ES GEHT WEITER, OHNE MICH”",NaN,cdfc5dc9-9f6d-7c1e-da35-572f2749508e,18 JAHRE MODERIERT RÖBI KOLLER «HAPPY DAY». NA...,"“ES GEHT WEITER, OHNE MICH”",no_link
9142,56538588,2025-04-03 00:00:00+02:00,TVS,TV Star,9420,"“WIR HATTEN EIN HAUS, KEINE VILLA”",NaN,72d9fdfe-bb00-7ffd-8561-d65738c658c0,HANS ROSENTHAL WAR EINE DER PRÄGENDSTEN PERSÖN...,"“WIR HATTEN EIN HAUS, KEINE VILLA”",no_link
9143,56550786,2025-04-04 00:00:00+02:00,URZ,Urner Zeitung,5166,† Rosemarie Gisler-Herger,NaN,f688de01-5b9f-5bbf-257c-a2e9c03f2cc9,Bürglen Der Spaziergang in die Feldgasse war f...,† Rosemarie Gisler-Herger,no_link
